In [38]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from IPython.display import clear_output
set_matplotlib_formats('retina')
#from tqdm import tqdm
#tqdm.pandas()
import numpy as np
from pyhive import presto
from datetime import datetime, timedelta
# from bson import ObjectId
from functools import reduce
from sklearn.cluster import KMeans
# from pymongo import MongoClient
import glob
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import json
import re
#import dtale
#from h3 import h3
# import pandasql as ps
import datetime

In [106]:
metabase_connection = presto.connect(
        # host='bi-trino-2.serving.data.production.internal',
#         host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
         host="processing-2.processing.data.production.internal",
         
#         host="presto.processing.yoda.run",
        # host="processing.processing.data.production.internal",


        port=80,
        protocol='http',
        catalog ='hive',
        username='abhishek.akki@rapido.bike')
    

In [113]:
startdate=pd.to_datetime('2025-01-01',format='%Y-%m-%d')
RedeemDataLoad=pd.DataFrame()
for i in range(10):
      q=f"""
            with tbl as(
                  select _id as redeemId,uniqueid,yyyymmdd,userdata_city,row_number() over(partition by uniqueid order by updated_epoch desc) as r,approver,
                  transform(CAST((json_extract(data, '$.orderData.ordersData')) AS ARRAY<JSON>), x -> json_extract_scalar(x, '$.orderId')) as order_id,
                  transform(CAST((json_extract(data, '$.orderData.ordersData')) AS ARRAY<JSON>), x -> json_extract_scalar(x, '$.distance')) as distance,
                  transform(CAST((json_extract(data, '$.orderData.ordersData')) AS ARRAY<JSON>), x -> json_extract_scalar(x, '$.serviceId')) as serviceId,
                  cast(json_extract(data, '$.orderData.finalIncentiveOrder') as array<json>) as finalIncentiveOrder,
                  array_join(transform(cast((json_extract(data,'$.failureReasons')) as array<json>),x->json_extract(x,'$.failureExpression')), ', ') as failureExpression,
                  transform(cast((json_extract(data,'$.failureReasons')) as array<json>),x->json_extract_scalar(x,'$.failureValue')) as failureValue,
                  from_unixtime(cast(lastredeemon as bigint)/1000,'Asia/Kolkata') as lastredeemon,status,tpstatus
                  from raw.kafka_redeem_transactions_immutable
                  where yyyymmdd='{startdate.strftime('%Y%m%d')}'
                  and (approver is null
                  or lower(approver) like '%dashboard%')
            )
            select * from tbl
            where r=1
      """
      print(startdate)
      df=pd.read_sql(q,metabase_connection)
      RedeemDataLoad=pd.concat([RedeemDataLoad,df],axis=0)
      startdate=startdate+timedelta(days=1)

2025-01-01 00:00:00
2025-01-02 00:00:00
2025-01-03 00:00:00
2025-01-04 00:00:00
2025-01-05 00:00:00
2025-01-06 00:00:00
2025-01-07 00:00:00
2025-01-08 00:00:00
2025-01-09 00:00:00
2025-01-10 00:00:00


In [114]:
RedeemDataLoad.to_csv('dashboardredemdata.csv')

In [102]:
str(columns)

"['_id', 'yyyymmdd', 'hhmmss', 'quarter_hour', 'epoch', 'updated_yyyymmdd', 'updated_hhmmss', 'updated_epoch', 'updated_quarter_hour', 'updated', 'eventname', 'eventid', 'eventtime', 'version', 'transactiontype', 'uniqueid', 'amount', 'type', 'userid', 'status', 'createdon', 'date', 'message', 'autoapproval', 'userdata', 'userdata_name', 'userdata_mobile', 'userdata_city', 'userdata_shift', 'lastredeemon', 'upidetails', 'upidetails_upiid', 'upidetails_name', 'approver', 'gateway', 'tpid', 'tpmessage', 'tpstatus', 'accountdetails', 'accountdetails_name', 'accountdetails_ifsc', 'accountdetails_accountno', 'accountdetails_branch', 'penaltydata', 'updatelogs', 'updatelog', 'updatelog_status', 'updatelog_userid', 'updatelog_message', 'updatelog_remarks', 'updatelog_regionalremarks', 'updatelog_regionalupdatetimestamp', 'updatelog_timestamp', 'updatelog_color', 'updatelog_penaltyamount', 'updatelog_level', 'updatelog_updatedby', 'updatelog_regionaluserid', 'updatelog_regionalupdatedby', 'ban

In [115]:
RedeemDataLoad.columns

Index(['redeemId', 'uniqueid', 'yyyymmdd', 'userdata_city', 'r', 'approver',
       'order_id', 'distance', 'serviceId', 'finalIncentiveOrder',
       'failureExpression', 'failureValue', 'lastredeemon', 'status',
       'tpstatus'],
      dtype='object')